In [1]:
#Cargando librerías 
import importlib

import matplotlib.pyplot as plt

from matplotlib import cm
import matplotlib as mpl
import itertools as it


#from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import scipy as sp
import pandas as pd

import numpy as np
import itertools as it
import random 

import os
import sys  


root = os.getcwd() + "/.."

sys.path.insert(0, root)

import src.positioning_sensors as ps
importlib.reload(ps)

import src.network_fitness as netfit
importlib.reload(netfit)

import src.manageExperiments as me
importlib.reload(me)

import config.config as cfg
importlib.reload(cfg)


<module 'config.config' from '/Users/rodrigo/Documents/SourcesCodes/GitHub/optimization_sensor_locations_with_constraints/notebooks/../config/config.py'>

In [2]:
n_extra_sensors = 2

#varbound = np.array([[0, 10], [0, 20]]*n_extra_sensors)

In [3]:
#cuantos sensores estamos dispuestos a mover
move_sensor = 2

In [4]:
fn = "linz_with_init_net.txt"
setup = me.readConfigFile(fn)

$$C^n_r$$

In [5]:
L = setup["init_fixed_net"]
n = len(L)
r = 3

In [6]:
#cargamos imagenes en luminance e importance
ilumina = setup["folder_input"] + setup["input_ntli"]
niveles = setup["folder_input"] + setup["input_evm"]
sigma_y = setup['filterg_sy']
sigma_x = setup['filterg_sx']
sigma = [sigma_y, sigma_x]

In [7]:
NTLI = ps.readIMG(ilumina)

In [8]:
ilumina = cfg.data + setup["folder_input"] + setup["input_ntli"]
niveles = cfg.data + setup["folder_input"] + setup["input_evm"]

#NTLI: luminance
NTLI = ps.readIMG(ilumina)

FDNTLI = sp.ndimage.filters.gaussian_filter(NTLI, 
                                            sigma, 
                                            mode = setup['gaussian_mode'])

/var/folders/k9/wpfgnz550_n_k3t9tg8scnlw0000gn/T/ipykernel_8030/1759838245.py:7: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  FDNTLI = sp.ndimage.filters.gaussian_filter(NTLI,


In [43]:
var_bound = list([NTLI.shape]*n)
c = len(var_bound)

In [44]:
def generateCombinations(n, r):
    elements = range(n)
    
    n = [x for x in range(n)]

    #all the possible combinations without repetition taken in groups  C_(n, x), 
    #n: total number of different elements
    #x: number of selection

    #all the combinations
    combination = list(it.combinations(n, r))

    return np.array(combination)
    

C = generateCombinations(5, r)

In [45]:
allc = setup["sensitivity_c"]
alls = setup["nsensors"]

In [46]:
EAM = ps.readIMG(niveles, invert = True)

EAM[EAM==254]=1.0

In [47]:
sensitivity = ps.f5(FDNTLI, EAM, c)

 W = i*a^(p)


In [48]:
#Loading precalculated local and directed empirical variograms
path2output_semivars = (cfg.experiment 
          + setup["experiment_id"]
          + setup["folder_semivariances"]
          + setup["output_semivariances"] 
                       )

In [49]:
FDNTLI = sp.ndimage.filters.gaussian_filter(NTLI, 
                                            sigma, 
                                            mode = setup['gaussian_mode'])

/var/folders/k9/wpfgnz550_n_k3t9tg8scnlw0000gn/T/ipykernel_8030/2836547934.py:1: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  FDNTLI = sp.ndimage.filters.gaussian_filter(NTLI,


In [50]:
filesv = path2output_semivars + ".csv"
filesvmask = path2output_semivars + "mask.csv"

nonsat = FDNTLI

In [51]:
data = pd.read_csv(filesv)
data_m = pd.read_csv(filesvmask)
variogram_set = np.array(data.iloc[:,3:])
variogram_set_m = np.array(data_m.iloc[:,3:])

In [52]:
coords = np.array(data.iloc[:,1:3])

In [53]:
variograms = variogram_set.reshape(len(variogram_set),nonsat.shape[0],nonsat.shape[1])

In [54]:
variograms_m = variogram_set_m.reshape(len(variogram_set),nonsat.shape[0],nonsat.shape[1])

In [55]:
aptitude = netfit.NetworkFitness(FDNTLI,
                                     EAM, 
                                     sensitivity, 
                                     variograms, 
                                     variograms_m,
                                     coords)

Selct cost functions: 
 'xor','max' or 'cover'


In [56]:
aptitude.selectFitnessFunction("max")
f = aptitude.f

In [57]:
aptitude.setCombinations(C)


In [58]:
var_bound.append((0,len(C)))

In [59]:
varbound = [[0, NTLI.shape[0]], [0, NTLI.shape[1]]]*n

In [60]:
varbound.append([0,len(C)-1]) 

In [62]:
#we select random combinations
combination = aptitude.c


In [63]:
random_pos = [random.randint(r[0], r[1]) for r in varbound]

In [64]:
Y = np.array(L).flatten()

In [65]:
X = random_pos[:-1]

random_com = random_pos[-1]

fp = np.array(L).flatten()

In [66]:
Z = combination[random_com]

In [76]:
Z

array([0, 1, 4])

In [67]:
vect_selection = np.zeros(len(L)*1)
vect_selection[Z] = 1

In [75]:
L

[[34, 34], [34, 31], [35, 29], [36, 39], [39, 22]]

In [68]:
W = np.array(L).flatten()

In [69]:
#We move selected sensors

In [70]:
for ix, x in enumerate(vect_selection):
    
    if x == 1:
        W[ix*2:ix*2+2] = X[ix*2:ix*2+2]

In [71]:
vect_selection

array([1., 1., 0., 0., 1.])

In [72]:
Y

array([34, 34, 34, 31, 35, 29, 36, 39, 39, 22])

In [73]:
W

array([ 0, 52, 13, 35, 35, 29, 36, 39, 19, 22])

In [74]:
X

[0, 52, 13, 35, 21, 59, 43, 56, 19, 22]